In [45]:
def taper_meridionally(data, taper_latitude):
    import numpy as np
    
    n_taper = np.shape(np.where(meridional_gridpoints/METERS_PER_DEGREE >= taper_latitude)[0])[0]
    taper_edge = np.cos(np.linspace(0, np.pi/2, n_taper))
    taper_edge[taper_edge < 1e-2] = 0
    taper_center = np.ones((ny-2*n_taper))
    taper = np.concatenate((taper_edge[::-1], taper_center, taper_edge))
    tapered_data = np.einsum(
        'ij,i->ij',
        data,
        taper
    )
    return tapered_data

def fringe_region_damping_function(latitude_grid, starting_latitude, ending_latitude, damping_width, damping_strength):
    """
    This function generates an array containing damping values as a function of latitude.
    
    Parameters
    ----------
    latitude_grid : array_like
        An array of grid points with dimensions of degrees latitude
    starting_latitude : float
        The latitude at which the damping should start, in degrees
    ending_latitude : float
        The latitude at which the damping should end, in degrees
    damping_width : float
        The width of the damping region, in degrees
    damping_strength : 
        The maximum value of the damping function
    
    Returns
    -------
    fringe_function : array_like
        The damping of the fringe region at each latitude
    """
    
    import numpy as np

    if np.max(np.abs(latitude_grid) > 90):
        raise ValueError("Latitude grid values must be between -90 and 90")

    ny = len(latitude_grid)
    def S(x):
        S = np.ones((ny))
        S[x<=0] = 0
        S[(0<x)*(x<1)] = 1/(1+np.exp(1/(x[(0<x)*(x<1)]-1) + 1/x[(0<x)*(x<1)])) 
        S[x>=1] = 1
        return S

    fringe_function = damping_strength*(1 - (
        + S((latitude_grid - starting_latitude)/damping_width)
        - S(1 + (latitude_grid - ending_latitude)/damping_width)
    ))
    
    return fringe_function